# Projet Final Apache Spark

**Nom Etudiant :**  DIALLO

**Prenom Etudiant:**  Ousmane

**Classe :**  Master 1 Big Data Analytics


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.2.15:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610275754814)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(
  StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [9]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

Dans cas de figure pour ne pas prendre en compte les valeurs null, on a utilisé la fonction **isNotNull**.
Ainsi on peut selectionner les lignes de la colonne "CallType" sans valeurs nulles et utiliser les fonctions **distinct** et **count** pour compter    les valeurs différentes et les affecter à la variable **callTypeCount**. 

In [11]:
// Reponse 1

val callTypeCount = fireDF
    .select("CallType")
    .filter(col("CallType").isNotNull)
    .distinct()
    .count()
callTypeCount

callTypeCount: Long = 30
res6: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

Dans cette cellule on fait comme à la précédente mais en enlèvant la fonction **count()** pour trouver seulement les valeurs de "CallType" difféntes
Ensuite on affiche le résultat avec la fonction **show** qui quant lui applique l'argument **false** affiche correctement les textes sans les abrèger. L'argument *30* est justifié par le fait qu'on a trouvé 30 types d'appels.

In [12]:
// Reponse 2

val callType = fireDF
    .select("CallType")
    .filter(col("CallType").isNotNull)
    .distinct()

callType.show(30, false)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Marine Fire                                 |
|Aircraft Emergency                          |
|Confined Space / Structure Collapse         |
|Administrative                              |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Watercraft in Distress                      |
|Explosion                                   |
|Oil Spill                                   |
|Vehicle Fire                                |
|Suspicious Package                          |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Assist Polic

callType: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

Après avoir eu la nouvelle base de données **newFireDF** avec le renommage de la colonne "Delay" en "ResponseDelayedinMins" on peut ainsi:<br>
 * créer une variable **responseDelayAboveFive** qui va recevoir les données retournées
 * Selectionner les colonnes ("CallNumber", "IncidentNumber", "ResponseDelayedinMins")
 * Appliquer un filtre et choisir les cas ou le delais de réponse est supérieur à 5 minutes avec le clause **where** <br>
Par soucis de présentation on affiche les 20 premières lignes du résultats avec la fonction **show** on peut tout de même afficher toutes les lignes avec la fonction **collect** : *responseDelayAboveFive.collect().foreach(println)*

In [13]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")

val responseDelayAboveFive = newFireDF
    .select("CallNumber", "IncidentNumber", "ResponseDelayedinMins")
    .where($"ResponseDelayedinMins" > 5)

responseDelayAboveFive.show(false)
//responseDelayAboveFive.collect().foreach(println)

+----------+--------------+---------------------+
|CallNumber|IncidentNumber|ResponseDelayedinMins|
+----------+--------------+---------------------+
|20110315  |2003409       |5.35                 |
|20120147  |2003642       |6.25                 |
|20130013  |2003818       |5.2                  |
|20140067  |2004152       |5.6                  |
|20140177  |2004216       |7.25                 |
|20150056  |2004408       |11.916667            |
|20150254  |2004521       |5.116667             |
|20150265  |2004528       |8.633333             |
|20150265  |2004528       |95.28333             |
|20150380  |2004610       |5.45                 |
|20150414  |2004641       |7.6                  |
|20160059  |2004721       |6.133333             |
|20160064  |2004724       |5.1833334            |
|20170118  |2005038       |6.9166665            |
|20170342  |2005173       |5.2                  |
|20180129  |2005321       |6.35                 |
|20180191  |2005353       |7.983333             |


newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
responseDelayAboveFive: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallNumber: int, IncidentNumber: int ... 1 more field]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [14]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res9: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

Pour trouver les types d'appels les plus courants on fait une projection sur la colonne "CallType" et ensuite on lui applique les conditions suivantes:
 * groupBy("CallType") pour aggrèger les type d'appel
 * count() pour compter le nombre de type d'appel
 * orderBy(desc("count")) pour ordonner le résultat suivant le plus grand nombre associé au type d'appel et de manière décroissante
 * limit(3) pour limiter le résultat au top 3 de la liste car leurs nombres est largement plus grands que les autres.

In [15]:
//Reponse 4

val mostCommonCalls = fireTSDF
    .select("CallType")
    .groupBy("CallType")
    .count()
    .orderBy(desc("count"))
    .limit(3)

mostCommonCalls.show(false)

+----------------+------+
|CallType        |count |
+----------------+------+
|Medical Incident|113794|
|Structure Fire  |23319 |
|Alarms          |19406 |
+----------------+------+



mostCommonCalls: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

Pour touver les boites postales rencontrées dans les appels les plus courants, on selectionne toutes les boites postales ensuite on fait un filtre suivant les types d'appels les plus courants qui sont "Medical Incident", "Structure Fire" et "Alarms"
Aussi par soucis de présentation on affiche que 20 lignes.

In [16]:
//Reponse 5-a

val mostCommonZip = fireTSDF
    .select("Zipcode")
    .distinct()
    .where($"CallType" === "Medical Incident" || 
           $"CallType" === "Structure Fire" || 
           $"CallType" === "Alarms")

mostCommonZip.show(false)
//mostCommonZip.collect().foreach(println)

+-------+
|Zipcode|
+-------+
|94109  |
|94115  |
|94112  |
|94127  |
|94108  |
|94121  |
|94105  |
|94131  |
|94134  |
|94124  |
|94102  |
|94114  |
|94111  |
|94103  |
|94117  |
|94122  |
|94110  |
|94132  |
|94104  |
|94133  |
+-------+
only showing top 20 rows



mostCommonZip: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Zipcode: int]


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

Pour répondre à cette question on fait comme suit:
   * selection des quartiers , des villes et des boites postales("Neighborhood", "Zipcode", "City")
   * chosir ceux de la ville de San Francisco et des boites postales "94102" ou "94103"

In [17]:
//Reponse 5-b

val sfNbh = fireTSDF
    .select("Neighborhood", "Zipcode", "City")
    .where($"City" === "San Francisco" && 
           ($"Zipcode" === 94102 || $"Zipcode" === 94103))

sfNbh.show(false)
//sfNbh.collect().foreach(println)

+----------------+-------+-------------+
|Neighborhood    |Zipcode|City         |
+----------------+-------+-------------+
|South of Market |94103  |San Francisco|
|Mission         |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|Tenderloin      |94102  |San Francisco|
|South of Market |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|South of Market |94103  |San Francisco|
|Mission         |94103  |San Francisco|
|Mission         |94103  |San Francisco|
|Tenderloin      |94102  |San Francisco|
|Tenderloin      |94102  |San Francisco|
|South of Market |94103  |San Francisco|
|Tenderloin      |94102  |San Francisco|
|Tenderloin      |94102  |San Francisco|
|South of Market |94103  |San Francisco|
|Western Addition|94102  |San Francisco|
|South of Market |94103  |San Francisco|
+----------------+-------+-------------+
only showing top

sfNbh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, Zipcode: int ... 1 more field]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

Pour afficher les statistiques sur une colonne d'un dataframe on l'applique la fonction describe suivi d'un show

Mais pour être plus précis ici on va le faire en choisissant les estimateurs que l'on nous demande d'afficher.
 * count: nous permet d'avoir le nombre total
 * mean: nous permet d'avoir la moyenne
 * min et max: nous donne le minimum et le maximum
 * round: nous permet d'arrondir le résultat et d'avoir un résultat plus commode à la lecture

In [18]:
//Reponse 6
fireTSDF.select(
                count("ResponseDelayedinMins").as("Nombes total d'appels"), 
                round(mean("ResponseDelayedinMins"), 3).as("La Moyenne"),
                round(min("ResponseDelayedinMins"), 3).as("Le Minimum"),
                max("ResponseDelayedinMins").as("Le Maximum"))
        .show(false)

//fireTSDF.describe("ResponseDelayedinMins").show(false)

+---------------------+----------+----------+----------+
|Nombes total d'appels|La Moyenne|Le Minimum|Le Maximum|
+---------------------+----------+----------+----------+
|175296               |3.892     |0.017     |1844.55   |
+---------------------+----------+----------+----------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

Pour connaitre le nombre d'années que l'on trouve dans le dataset on choisit de se projeter sur la colonne "IncidentDate" car cette colonne marque l'activité du service puisque les appels déclenchent un enregistrement.
Ensuite on compte le nombre d'années différentes que l'on extrait des dates avec la fontion de __year()__

In [76]:
//Reponse 7-a

val nbOfYears = fireTSDF
    .select(year(col("IncidentDate")))
    .distinct()
    .count()

nbOfYears

nbOfYears: Long = 19
res47: Long = 19


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

Pour trouver la semaine de l'année 2018 qui a eu le plus d'apels d'incendie on procède comme suit:
 * On cible l'année de l'incident qui est 2018
 * On cible les appels dont le type ou le nom contient le mot "fire(feu ou incendie)" qui est le mot clé de la recherche 
 * On applique la fonction **lower** pour transformer la valeur de la colonne en miniscule car le mot clé est en miniscule
 * On se projette sur les semaines correspondant à ces appels, ensuite on les ordonnes par fréquence et on choisit le plus fréquent

In [46]:
//Reponse 7-b

val hottestWeek = fireTSDF
    .where(year(col("IncidentDate")) === 2018)
    .filter(lower(col("CallType")).like("%fire%"))
    .select(weekofyear(col("IncidentDate")).as("Fire Week"))
    .groupBy("Fire Week")
    .count()
    .orderBy(desc("count"))
    .limit(1)
   
hottestWeek.show()

+---------+-----+
|Fire Week|count|
+---------+-----+
|        1|   37|
+---------+-----+



hottestWeek: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Fire Week: int, count: bigint]


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

Pour répondre à cette question nous allons:
 * d'abord cibler la ville de San Francisco et l'année 2018
 * ensuite, on fait une projection sur les colonnes "Neighborhood", "ResponseDelayedinMins"
 * enfin on fait une aggrégation suivant les quartiers et la moyenne des temps de réponse de chacun que l'on ordonne de manière décroissante

In [74]:
//Reponse 8

val worserDelay = fireTSDF
    .where($"City" === "San Francisco")
    .where(year(col("IncidentDate")) === 2018)
    .select("Neighborhood", "ResponseDelayedinMins")
    .groupBy("Neighborhood")
    .agg(avg("ResponseDelayedinMins").as("Average Delay"))
    .orderBy(desc("Average Delay"))
    
worserDelay.show(5, false) //on a choisi d'afficher le top 5.

+---------------------+------------------+
|Neighborhood         |Average Delay     |
+---------------------+------------------+
|Treasure Island      |11.320833250880241|
|Presidio             |6.248148073752721 |
|Chinatown            |6.158818309742307 |
|McLaren Park         |4.74404764175415  |
|Bayview Hunters Point|4.629759605458373 |
+---------------------+------------------+
only showing top 5 rows



worserDelay: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, Average Delay: double]


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [77]:
//Reponse 9

fireTSDF.
    write.
    mode("overwrite").
    option("compression", "none").
    parquet("datasets/project/fireTSDF")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [78]:
//Reponse 10

spark.read.parquet("datasets/project/fireTSDF").printSchema

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

In [79]:
//Reponse 10
spark.read.parquet("datasets/project/fireTSDF").printSchema

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 |-- SupervisorDistrict: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Location: string (nullable =

## FIN